In [2]:
# Import necessary libraries
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from alpaca_trade_api import REST
from datetime import datetime, timedelta
import pytz

# Load environment variables from .env file
load_dotenv(dotenv_path='APCA.env')  # Ensure this matches the location of your .env file

# Define monthly income
monthly_income = 12000

# ---- Evaluate Cryptocurrency Wallet ----
# Define cryptocurrency amounts
btc_amount = 1.2
eth_amount = 5.3

# Fetch current cryptocurrency prices using CoinGecko API
btc_url = 'https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd'
eth_url = 'https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd'

btc_response = requests.get(btc_url).json()
eth_response = requests.get(eth_url).json()

btc_price = btc_response['bitcoin']['usd']
eth_price = eth_response['ethereum']['usd']

# Calculate total cryptocurrency wallet value
btc_value = btc_price * btc_amount
eth_value = eth_price * eth_amount
total_crypto_value = btc_value + eth_value

# ---- Evaluate Stock and Bond Holdings ----
# Set Alpaca API key and secret from .env file
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
alpaca_api_endpoint = os.getenv('ALPACA_API_ENDPOINT')

# Initialize Alpaca REST API
api = REST(alpaca_api_key, alpaca_secret_key, base_url=alpaca_api_endpoint, api_version='v2')

# Define the portfolio parameters
tickers = ["SPY", "AGG"]
timeframe = "1D"

# Define the date range for historical data
start_date = pd.Timestamp('2013-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-01-01', tz='America/New_York').isoformat()

# Fetch historical data
portfolio_df = api.get_bars(tickers, timeframe, start=start_date, end=end_date).df

# Process historical data for SPY and AGG
portfolio_df = portfolio_df.unstack(level=0).pivot(columns='symbol', values='close_price')

# Calculate daily returns
daily_returns = portfolio_df.pct_change().dropna()

# Define the simulation parameters
num_simulations = 500
num_trading_days = 252 * 30  # 30 years

# Initialize empty DataFrame to store simulation results
all_simulations = pd.DataFrame()

# Simulate the portfolio performance over 30 years
for n in range(num_simulations):
    simulated_prices = pd.DataFrame()
    weights = np.array([0.6, 0.4])  # 60/40 stocks to bonds ratio for 30 years simulation
    for ticker in tickers:
        prices = (1 + np.random.normal(daily_returns[ticker].mean(), daily_returns[ticker].std(), num_trading_days)).cumprod()
        simulated_prices[ticker] = prices
    portfolio_simulated = (simulated_prices * weights).sum(axis=1)
    all_simulations[n] = portfolio_simulated

# Plot the simulations
plt.figure(figsize=(10,6))
plt.plot(all_simulations)
plt.title('30 Years Portfolio Performance Simulations')
plt.xlabel('Days')
plt.ylabel('Portfolio Cumulative Returns')
plt.show()

# Calculate the 95% confidence interval for the ending returns
ci_lower = all_simulations.iloc[-1].quantile(0.025)
ci_upper = all_simulations.iloc[-1].quantile(0.975)

# Print the confidence interval
print(f"There is a 95% chance that an initial investment in the portfolio over the next 30 years will end within in the range of ${ci_lower:,.2f} and ${ci_upper:,.2f}")

# Adjust the weights for the 10-year simulation to 80% stocks and 20% bonds
weights_10_year = np.array([0.8, 0.2])

# Adjust the number of trading days for a 10-year horizon
num_trading_days_10_year = 252 * 10

# Initialize empty DataFrame to store the 10-year simulation results
simulation_10_year = pd.DataFrame()

# Run the Monte Carlo Simulation for 10 years
for n in range(num_simulations):
    simulated_prices_10_year = pd.DataFrame()
    for ticker in tickers:
        # Use the historical daily returns to generate simulated daily returns
        simulated_daily_returns = np.random.normal(daily_returns[ticker].mean(), daily_returns[ticker].std(), num_trading_days_10_year)
        simulated_prices = (1 + simulated_daily_returns).cumprod()
        simulated_prices_10_year[ticker] = simulated_prices
    # Calculate the portfolio daily returns with the adjusted weights
    portfolio_simulated_10_year = (simulated_prices_10_year * weights_10_year).sum(axis=1)
    simulation_10_year[n] = initial_investment * portfolio_simulated_10_year

# Plot the 10-year simulation outcomes
plt.figure(figsize=(10,6))
plt.plot(simulation_10_year)
plt.title('10 Years Portfolio Performance Simulations with 80/20 Stocks/Bonds Allocation')
plt.xlabel('Days')
plt.ylabel('Portfolio Cumulative Returns')
plt.show()

# Calculate the 95% confidence interval for the ending returns over 10 years
ci_lower_10_year = simulation_10_year.iloc[-1].quantile(0.025)
ci_upper_10_year = simulation_10_year.iloc[-1].quantile(0.975)

# Print the confidence interval for the 10-year simulation
print(f"There is a 95% chance that an initial investment in the portfolio over the next 10 years will end within in the range of ${ci_lower_10_year:,.2f} and ${ci_upper_10_year:,.2f}")

ValueError: Columns with duplicate values are not supported in stack